In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import io
import glob
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import numpy as np
import re

## Change to your directory holding all the images 

In [2]:
image6Directory = '/Users/spsither/Desktop/MS Thesis/GrandPass_Jan08/Images6/'

In [3]:
imageFiles = glob.glob(image6Directory + r'*.JPG')
imageFiles.sort()

In [7]:
import sys

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_exif_data(image):
    """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
    exif_data = {}
    info = image._getexif()
    if info:
        for tag, value in info.items():
            decoded = TAGS.get(tag, tag)
            if decoded == "GPSInfo":
                gps_data = {}
                for gps_tag in value:
                    sub_decoded = GPSTAGS.get(gps_tag, gps_tag)
                    gps_data[sub_decoded] = value[gps_tag]

                exif_data[decoded] = gps_data
            else:
                exif_data[decoded] = value

    return exif_data

def _convert_to_degree(value):
    """Helper function to convert the GPS coordinates stored in the a to degress in float format"""
    d = value[0]
    m = value[1]
    s = value[2]
    return d + (m / 60.0) + (s / 3600.0)

def get_lat_lon(exif_data):
    """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
    lat = None
    lon = None

    if "GPSInfo" in exif_data:
        gps_info = exif_data["GPSInfo"]

        gps_latitude = gps_info.get("GPSLatitude")
        gps_latitude_ref = gps_info.get('GPSLatitudeRef')
        gps_longitude = gps_info.get('GPSLongitude')
        gps_longitude_ref = gps_info.get('GPSLongitudeRef')

        if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
            lat = _convert_to_degree(gps_latitude)
            if gps_latitude_ref != "N":                     
                lat *= -1

            lon = _convert_to_degree(gps_longitude)
            if gps_longitude_ref != "E":
                lon *= -1

    return lat, lon
def get_date_time(exif_data):
    date_time = None
    if "DateTime" in exif_data:
        date_time = exif_data["DateTime"]
    return date_time

def get_gimbal_degree(exif_data):
    gimbal_degree = None
    if "MakerNote" in exif_data:
        makerNote = str(exif_data["MakerNote"])
        matchGimbalDeg = re.search( r'GimbalDegree\(Y,P,R\):([-+]?\d*),([-+]?\d*),([-+]?\d*)', makerNote)
        if matchGimbalDeg:
            gimbal_degree = matchGimbalDeg.group(1),matchGimbalDeg.group(2),matchGimbalDeg.group(3)
            gimbal_degree = list(map(float, gimbal_degree))
    return gimbal_degree

def get_flight_degree(exif_data):
    flight_degree = None
    if "MakerNote" in exif_data:
        makerNote = str(exif_data["MakerNote"])
        matchFlightDeg = re.search( r'FlightDegree\(Y,P,R\):([-+]?\d*),([-+]?\d*),([-+]?\d*)', makerNote)
        if matchFlightDeg:            
            flight_degree = matchFlightDeg.group(1),matchFlightDeg.group(2),matchFlightDeg.group(3)
            flight_degree = list(map(float, flight_degree))
    return flight_degree


################
# Example ######
################

# load an image through PIL's Image object

image = Image.open(imageFiles[0])
exif_data = get_exif_data(image)
print('Image file name: ',imageFiles[0])
print('Latitude and Longitude of the image: ', get_lat_lon(exif_data))
print('Date time of the image: ', get_date_time(exif_data))
print('Gimbal degree (Yaw, Pitch, Roll): ', get_gimbal_degree(exif_data))
print('Drone degree (Yaw, Pitch, Roll): ',get_flight_degree(exif_data))

Image file name:  /Users/spsither/Desktop/MS Thesis/GrandPass_Jan08/Images6/F1_DJI_0109.JPG
Latitude and Longitude of the image:  (39.30295294444444, -93.32649124999999)
Date time of the image:  2021:01:08 12:50:32
Gimbal degree (Yaw, Pitch, Roll):  [827.0, -899.0, 0.0]
Drone degree (Yaw, Pitch, Roll):  [867.0, 4.0, -58.0]
